#### For Colab

In [1]:
"""
function ClickConnect(){
    console.log("Working");
    document.querySelector("colab-toolbar-button").click() 
}
var i = setInterval(ClickConnect, 900000)
clearInterval(i)
"""

'\nfunction ClickConnect(){\n    console.log("Working");\n    document.querySelector("colab-toolbar-button").click() \n}\nvar i = setInterval(ClickConnect, 900000)\nclearInterval(i)\n'

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
drive_dir = '/content/drive/My Drive/MAGMA: Summarization/'

#### Install Libraries

In [4]:
requirements_dir = (drive_dir + 'transformers/examples/seq2seq/').replace(' ', '\ ')
requirements_file = requirements_dir + 'requirements.txt'
!cd $requirements_dir; pip install -r $requirements_file

!pip install transformers==4.1.1
!pip install -U pyarrow
!pip install -U wandb

     |████████████████████████████████| 51kB 6.0MB/s 
     |████████████████████████████████| 71kB 7.7MB/s 
     |████████████████████████████████| 7.9MB 44.3MB/s 
     |████████████████████████████████| 7.5MB 46.7MB/s 
     |████████████████████████████████| 327kB 55.8MB/s 
     |████████████████████████████████| 163kB 59.6MB/s 
     |████████████████████████████████| 81kB 11.3MB/s 
     |████████████████████████████████| 1.2MB 45.2MB/s 
     |████████████████████████████████| 163kB 54.7MB/s 
     |████████████████████████████████| 81kB 12.5MB/s 
     |████████████████████████████████| 4.5MB 49.3MB/s 
     |████████████████████████████████| 112kB 55.5MB/s 
     |████████████████████████████████| 245kB 56.4MB/s 
     |████████████████████████████████| 71kB 11.0MB/s 
     |████████████████████████████████| 122kB 58.7MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=eb4ce93d222e3eb7bc895981a4ab163d03ef6bff51ba70f333df564ce1989f61
  Stored in di

## **Fine-tuning**

In [5]:
finetune_script = '"'+drive_dir+'transformers/examples/seq2seq/finetune_trainer.py"'
eval_script = '"'+drive_dir+'transformers/examples/seq2seq/run_eval.py"'

### **Config**

In [6]:
import sys
sys.path.insert(0, drive_dir)
import config

import torch
torch.manual_seed = config.SEED

import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

### XSUM

In [ ]:
data_dir = '"/content/drive/My Drive/MAGMA: Summarization/datasets/xsum"'

output_dir = '"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/bart_xsum_train"'
log_dir = output_dir + '/logs'

##### Fine tune

In [ ]:
#--warmup_steps 5
#--gradient_accumulation_steps 128
#--do_predict --n_test 20
#--dataloader_drop_last

In [ ]:
!python3 $finetune_script \
--model_name_or_path facebook/bart-large-cnn \
--tokenizer_name facebook/bart-large-cnn \
--data_dir $data_dir \
--fp16 \
--learning_rate 3e-5 --label_smoothing 0.1 \
--sortish_sampler --freeze_embeds --adafactor \
--task summarization \
--max_source_length 1024 \
--max_target_length 60 \
--do_train \
--n_train 80 --num_train_epochs 2 \
--logging_steps 10 --save_steps 40 \
--per_device_train_batch_size 2 --per_device_eval_batch_size 2 \
--do_eval --n_val 20 --evaluation_strategy epoch --val_max_target_length 60 --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--predict_with_generate \
--test_max_target_length 100 \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED

2021-01-05 10:53:25.802661: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/05/2021 10:53:29 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
01/05/2021 10:53:29 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/content/drive/My Drive/MAGMA: Summarization/fine-tuning/bart_xsum_train', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=2.0, max_steps=-1, warmup_steps=0, 

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/bart_test_xsum.txt"'
score_dir = output_dir[:-1] + '/bart_test_xsum.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--n_obs 20 --bs 2 \
--dump-args

2021-01-04 15:15:39.475338: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
100% 10/10 [00:40<00:00,  4.05s/it]
{'rouge1': 27.8322, 'rouge2': 10.8571, 'rougeL': 21.4738, 'rougeLsum': 23.8121, 'n_obs': 20, 'runtime': 40, 'seconds_per_sample': 2.0}


### Karger Books CC

In [ ]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [ ]:
data_dir = '"/content/drive/My Drive/MAGMA: Summarization/datasets/karger_books_chunk_chapter"'

output_dir = '"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_karger_books_cc_train"'
log_dir = output_dir + '/logs'

##### Fine tune

In [ ]:
!python3 $finetune_script \
--model_name_or_path facebook/bart-large-cnn \
--tokenizer_name facebook/bart-large-cnn \
--data_dir $data_dir \
--learning_rate 3e-5 --label_smoothing 0.1 \
--sortish_sampler --freeze_embeds --adafactor \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.BULLETS_MAX_LEN \
--val_max_target_length $config.BULLETS_MAX_LEN \
--test_max_target_length $config.BULLETS_MAX_LEN \
--do_train \
--logging_steps 10 --save_steps 200 --save_total_limit 3 \
--per_device_train_batch_size 2 --per_device_eval_batch_size 2 \
--do_eval --evaluation_strategy epoch --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED

2021-01-05 09:35:34.078850: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/05/2021 09:35:39 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
01/05/2021 09:35:39 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/content/drive/My Drive/MAGMA: Summarization/fine-tuning/bart_karger_books_moc_train', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, war

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_cc.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_cc.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--bs 2 \
--length_penalty $config.LENAGTH_PENALTY \
--no_repeat_ngram_size $config.NO_REPEAT_NGRAM_SIZE \
--num_beams $config.NUM_BEAMS \
--dump-args

### Karger Books MOC

In [ ]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [ ]:
data_dir = '"/content/drive/My Drive/MAGMA: Summarization/datasets/karger_books_moc"'

output_dir = '"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_karger_books_moc_train"'
log_dir = output_dir + '/logs'

##### Fine tune

In [ ]:
!python3 $finetune_script \
--model_name_or_path facebook/bart-large-cnn \
--tokenizer_name facebook/bart-large-cnn \
--data_dir $data_dir \
--learning_rate 3e-5 --label_smoothing 0.1 \
--sortish_sampler --freeze_embeds --adafactor \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.BULLETS_MAX_LEN \
--val_max_target_length $config.BULLETS_MAX_LEN \
--test_max_target_length $config.BULLETS_MAX_LEN \
--do_train \
--logging_steps 10 --save_steps 200 --save_total_limit 3 \
--per_device_train_batch_size 2 --per_device_eval_batch_size 2 \
--do_eval --evaluation_strategy epoch --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED

2021-01-05 09:35:34.078850: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/05/2021 09:35:39 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
01/05/2021 09:35:39 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/content/drive/My Drive/MAGMA: Summarization/fine-tuning/bart_karger_books_moc_train', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.EPOCH: 'epoch'>, prediction_loss_only=False, per_device_train_batch_size=2, per_device_eval_batch_size=2, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=3.0, max_steps=-1, war

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_moc.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_moc.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--bs 2 \
--length_penalty $config.LENAGTH_PENALTY \
--no_repeat_ngram_size $config.NO_REPEAT_NGRAM_SIZE \
--num_beams $config.NUM_BEAMS \
--dump-args

2021-01-05 10:47:49.980394: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
parsed the following generate kwargs: {'num_beams': 2}
100% 90/90 [03:31<00:00,  2.35s/it]
{'rouge1': 37.4339, 'rouge2': 14.2967, 'rougeL': 23.0324, 'rougeLsum': 34.7576, 'n_obs': 179, 'runtime': 211, 'seconds_per_sample': 1.1788, 'num_beams': 2}


### Karger Books Para

In [7]:
model_name_or_path = 'sshleifer/distilbart-cnn-12-6'

In [8]:
data_dir = '"/content/drive/My Drive/MAGMA: Summarization/datasets/karger_books_para/bart"'

output_dir = '"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_karger_books_para_train"'

log_dir = output_dir + '/logs'

In [9]:
from transformers import AutoConfig
model_config = AutoConfig.from_pretrained(model_name_or_path)
model_config.min_length = config.ONE_BULLET_MIN_LEN
model_config.max_length = config.ONE_BULLET_MAX_LEN
model_config.length_penalty = config.LENGTH_PENALTY
model_config.no_repeat_ngram_size = config.NO_REPEAT_NGRAM_SIZE

model_config.task_specific_params['summarization']['min_length'] = config.ONE_BULLET_MIN_LEN
model_config.task_specific_params['summarization']['max_length'] = config.ONE_BULLET_MAX_LEN
model_config.task_specific_params['summarization']['length_penalty'] = config.LENGTH_PENALTY
model_config.task_specific_params['summarization']['no_repeat_ngram_size'] = config.NO_REPEAT_NGRAM_SIZE
model_config_dir = '"/content/drive/My Drive/MAGMA: Summarization/fine-tuning/'+\
    model_name_or_path.replace('/', '?')+'_config"'
model_config.save_pretrained(model_config_dir[1:-1])

##### Fine tune

In [ ]:
!python3 $finetune_script \
--model_name_or_path $model_name_or_path \
--config_name $model_config_dir \
--tokenizer_name $model_name_or_path \
--data_dir $data_dir \
--fp16 \
--learning_rate 3e-5 --label_smoothing 0.1 \
--sortish_sampler --freeze_embeds --adafactor \
--task summarization \
--max_source_length 1024 \
--max_target_length $config.ONE_BULLET_MAX_LEN \
--val_max_target_length $config.ONE_BULLET_MAX_LEN \
--test_max_target_length $config.ONE_BULLET_MAX_LEN \
--do_train \
--num_train_epochs 10 \
--logging_steps 10 --save_steps 1000 --save_total_limit 3 \
--per_device_train_batch_size 4 --per_device_eval_batch_size 4 \
--do_eval --evaluation_strategy steps --eval_steps 1000 --eval_beams 2 \
--metric_for_best_model rougeL --greater_is_better True \
--predict_with_generate \
--output_dir $output_dir \
--overwrite_output_dir \
--seed $config.SEED \
--run_name $output_dir \

2021-01-13 15:46:33.581020: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
01/13/2021 15:46:36 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: True
01/13/2021 15:46:36 - INFO - __main__ -   Training/evaluation parameters Seq2SeqTrainingArguments(output_dir='/content/drive/My Drive/MAGMA: Summarization/fine-tuning/sshleifer?distilbart-cnn-12-6_karger_books_para_train', overwrite_output_dir=True, do_train=True, do_eval=True, do_predict=False, model_parallel=False, evaluation_strategy=<EvaluationStrategy.STEPS: 'steps'>, prediction_loss_only=False, per_device_train_batch_size=4, per_device_eval_batch_size=4, per_gpu_train_batch_size=None, per_gpu_eval_batch_size=None, gradient_accumulation_steps=1, eval_accumulation_steps=None, learning_rate=3e-05, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epoc

##### Evaluate

In [ ]:
source_test_dir = data_dir[:-1] + '/test.source"'
reference_test_dir = data_dir[:-1] + '/test.target"'

save_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.txt"'
score_dir = output_dir[:-1] + '/'+model_name_or_path.replace('/', '?')+'_test_karger_books_para.json"'

In [ ]:
!python3 $eval_script \
$output_dir \
$source_test_dir \
$save_dir \
--reference_path $reference_test_dir \
--score_path $score_dir \
--task summarization \
--bs 2 \
--length_penalty $config.LENAGTH_PENALTY \
--no_repeat_ngram_size $config.NO_REPEAT_NGRAM_SIZE \
--num_beams $config.NUM_BEAMS \
--dump-args

Unknown option: --
usage: python3 [option] ... [-c cmd | -m mod | file | -] [arg] ...
Try `python -h' for more information.
